In [1]:
import pandas as pd
import numpy as np

import glob, re, os

In [2]:
LR_events = np.sort(glob.glob('/Users/67981492/Desktop/loki_1/data/BIDS/sub-0811/ses-*/beh/*task-lokicat*.tsv'))
onsets = np.sort(glob.glob('/Users/67981492/Desktop/loki_1/data/BIDS/sub-0811/ses-*/func/*task-lokicat*events.tsv'))

In [3]:
n_runs = 5
n_sessions = 9 

assert len(LR_events) == (n_runs*n_sessions), 'check events files'
assert len(onsets) == (n_runs*n_sessions), 'check events files'

In [4]:
for LR_event, onset in zip(LR_events, onsets):

    LR_events_subject = LR_event.partition('sub-0')[2][:3]
    LR_events_session_n = LR_event.partition('ses-')[2][:2]
    LR_events_run_n = LR_event.partition('run')[2][:2]
    
    onset_subject = onset.partition('sub-0')[2][:3]
    onset_session_n = onset.partition('ses-')[2][:2]
    onset_run_n = onset.partition('run')[2][:2]
    
    assert (LR_events_subject, LR_events_session_n, LR_events_run_n) == (onset_subject, onset_session_n, onset_run_n)    

    events_path = ('/Users/67981492/Desktop/loki_1/data/BOLD/sub-{}/fmriprep/sub-{}/ses-{}/func/').format(onset_subject, onset_subject, onset_session_n) 

    if not os.path.exists(events_path):
        os.makedirs(events_path)
    
    LR_events_df = pd.read_csv(LR_event,sep=r'\,|\t',engine='python').rename(columns=lambda x: x.strip())
    onsets_df = pd.read_csv(onset,sep=r'\,|\t',engine='python').rename(columns=lambda x: x.strip())
    
    
    LR_event_design_df = pd.DataFrame()


    LR_event_design_df['reward_code'] = LR_events_df['reward_code'].copy() 
    LR_event_design_df['subj_id'] = LR_events_df['subj_id'].copy() 
    LR_event_design_df['condition'] = LR_events_df['condition'].copy() 
    LR_event_design_df['trial'] = LR_events_df['trial'].copy() 
    LR_event_design_df['epoch_trial'] = LR_events_df['epoch_trial'].copy() 
    LR_event_design_df['epoch_number'] = LR_events_df['epoch_number'].copy() 


    LR_event_design_df['LR'] = LR_events_df['LR_choice'].copy() 
    LR_event_design_df['onset'] = onsets_df['rt'].copy() + onsets_df['stim_onset'].copy()

    
    LR_event_design_df['session_n'] = int(LR_events_session_n)
    LR_event_design_df['run_n'] = int(LR_events_run_n)
    
    
    fn = os.path.join(events_path, os.path.basename(onset).partition('_events')[0] + '_LR_events.tsv')
    
    
    LR_event_design_df.to_csv(fn, index=False)